# Introduction to Delta Lake

Delta Lake is an open source project that enables building a Lakehouse architecture on top of data lakes. Delta Lake provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing on top of existing data lakes, such as S3, ADLS, GCS, and HDFS.

Specifically, Delta Lake offers:

* ACID transactions on Spark: Serializable isolation levels ensure that readers never see inconsistent data.
* Scalable metadata handling: Leverages Spark distributed processing power to handle all the metadata for petabyte-scale tables with billions of files at ease.
* Streaming and batch unification: A table in Delta Lake is a batch table as well as a streaming source and sink. Streaming data ingest, batch historic backfill, interactive queries all just work out of the box.
* Schema enforcement: Automatically handles schema variations to prevent insertion of bad records during ingestion.
* Time travel: Data versioning enables rollbacks, full historical audit trails, and reproducible machine learning experiments.
* Upserts and deletes: Supports merge, update and delete operations to enable complex use cases like change-data-capture, slowly-changing-dimension (SCD) operations, streaming upserts, and so on.
* Vibrant connector ecosystem: Delta Lake has connectors read and write Delta tables from various data processing engines like Apache Spark, Apache Flink, Apache Hive, Apache Trino, AWS Athena, and more.


## Data Warehouseing, Data Lake, and Data Lakehouse

### Data Warehouse

### Data Lake

### Data Lakehosue

## What is Delta Lake

### Common Use Cases

### Key Features

## Anatomy of a Delta Lake Table